# Modules
LangChain provides standard, extendable interfaces and external integrations for the following main modules:

## Model I/O
Interface with language models

## Retrieval
Interface with application-specific data

## Agents
Let chains choose which tools to use given high-level directives
> 아직은 실험적인 기술로 자동으로 선택되게 만듬

# Additional
## Chains
Common, building block compositions

## Memory
Persist application state between runs of a chain

## Callbacks
Log and stream intermediate steps of any chain
> 모델이 어떤 과정을 거치고 있는지 지켜볼 수 있도록 

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
)


t = PromptTemplate.from_template("What is the capital of {country}")
prompt = t.format(country= "France")
print(prompt)

What is the capital of France


In [9]:
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"]
)
prompt = t.format(country= "France")
print(prompt)


What is the capital of France


## FewShot 
> 프롬프트에 예제들을 준다는것

### FewShotPromptTemplate

In [11]:
examples = [
        {
            "question": "What do you know about France?",
            "answer": """,
            Here is what I know:,
            Capital: Paris,
            Language: French,
            Food: Wine and Cheese,
            Currency: Euro,
            """
        },
        {
            "question": "What do you know about Italy?",
            "answer": """,
            I know this:,
            Capital: Rome,
            Language: Italian,
            Food: Pizza and Pasta,
            Currency: Euro,
            """,
        },
        {
            "question": "What do you know about Greece?",
            "answer": """,
            I know this:,
            Capital: Athens,
            Language: Greek,
            Food: Souvlaki and Feta Cheese,
            Currency: Euro,
            """,
        },
    ]

In [12]:
chat.predict("What do you about France")

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. France is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

The country has a diverse landscape, ranging from beautiful coastlines along the Mediterranean Sea and the Atlantic Ocean to picturesque countryside and the majestic Alps. France is also renowned for its cuisine, including dishes like croissants, baguettes, cheese, wine, and gourmet cuisine.

Paris, the capital city of France, is a major global center for art, fashion, and gastronomy. It is home to numerous world-class museums, fashion houses, and Michelin-starred restaurants. Other notable cities in France include Marseille, Lyon, Toulouse, and Nice.

France is a member of the European Union and has a developed economy. It is known for its strong aerospace, automotive, luxury goods, and tourism industries. The country also has a well-d

'France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. France is famous for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nThe country has a diverse landscape, ranging from beautiful coastlines along the Mediterranean Sea and the Atlantic Ocean to picturesque countryside and the majestic Alps. France is also renowned for its cuisine, including dishes like croissants, baguettes, cheese, wine, and gourmet cuisine.\n\nParis, the capital city of France, is a major global center for art, fashion, and gastronomy. It is home to numerous world-class museums, fashion houses, and Michelin-starred restaurants. Other notable cities in France include Marseille, Lyon, Toulouse, and Nice.\n\nFrance is a member of the European Union and has a developed economy. It is known for its strong aerospace, automotive, luxury goods, and tourism industries. The country also has a

In [17]:
example_template = """
    Human : {question}
    AI : {answer}
"""
example_propmt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_propmt,
    examples=examples,
    suffix="Human : What do you know aboud {country}?",
    input_variables=["country"]
)
message = prompt.format(country="Germary")

for item in message.split("\n"):
    print(item)



    Human : What do you know about France?
    AI : ,
            Here is what I know:,
            Capital: Paris,
            Language: French,
            Food: Wine and Cheese,
            Currency: Euro,
            



    Human : What do you know about Italy?
    AI : ,
            I know this:,
            Capital: Rome,
            Language: Italian,
            Food: Pizza and Pasta,
            Currency: Euro,
            



    Human : What do you know about Greece?
    AI : ,
            I know this:,
            Capital: Athens,
            Language: Greek,
            Food: Souvlaki and Feta Cheese,
            Currency: Euro,
            


Human : What do you know aboud Germary?


In [19]:
chain = prompt | chat

chain.invoke({
    # "country" : "Germany"
    "country" : "Korea"
})

AI : 
I know this:
Capital: Seoul,
Language: Korean,
Food: Kimchi and Bibimbap,
Currency: South Korean won.

AIMessageChunk(content='AI : \nI know this:\nCapital: Seoul,\nLanguage: Korean,\nFood: Kimchi and Bibimbap,\nCurrency: South Korean won.')

### FewShotChatMessagePromptTemplate

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [5]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)


In [6]:
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        # 전문가라고만 하면 길게 답변을 하기에  you give short answers.
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum Soup
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum Soup\n        Currency: Thai Baht\n        ')

## Example Selector

In [2]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1, 
    streaming=True, 
    callbacks=[StreamingStdOutCallbackHandler()]
)


t = PromptTemplate.from_template("What is the capital of {country}")
prompt = t.format(country= "France")
print(prompt)

What is the capital of France


In [5]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]
example_prompt = PromptTemplate.from_template("Human:{question}\nAI:{answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=10,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

print(prompt)

KeyError: 'input_variables'